# Simple Ensembling

In [1]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 67.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 20.0 MB/s eta 0:00:00


In [2]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.7 MB/s eta 0:00:00


In [3]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.9 MB/s eta 0:00:00


In [4]:
from sklearn.ensemble import StackingClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from transformers import DistilBertForSequenceClassification, XLNetForSequenceClassification, DistilBertTokenizer, XLNetTokenizer
import torch
import numpy as np
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
# Load pretrained models
distilbert_model = DistilBertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/Models/DistilBERT/politifact_epoch_nlr_7c")
xlnet_model = XLNetForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/Models/XLNet/politifact_epoch7c")


In [7]:
# Tokenizers
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-cased")
xlnet_tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")


In [8]:
# Load dataset
df = pd.read_csv("/content/gdrive/MyDrive/fake_news/FakeNewsNet_DS/politifact_cleaned.csv")
df['SECTION_CLEANED'] = df['SECTION_CLEANED'].astype(str)

# Randomly select  % of the data
df_data = df.sample(frac=1.0, random_state=42)


# Split the dataset into training, validation, and test sets
# First, split into training and temp sets (80% training, 20% temp)
train_df, temp_df = train_test_split(df_data, test_size=0.20, random_state=42)

# Then, split the temp set into validation and test sets (50% validation, 50% test)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Optionally,reset the index of the DataFrames
train_df = train_df.reset_index(drop=True)
#validation_df = validation_df1.sample(frac=0.50, random_state=42)

validation_df = validation_df.reset_index(drop=True)

#test_df = test_df1.sample(frac=0.50, random_state=42)
test_df = test_df.reset_index(drop=True)

In [9]:
y_val = list(validation_df['label'])
len(y_val)

106

In [10]:
X_val = list(validation_df['SECTION_CLEANED'])

In [11]:
# Prepare input data
def preprocess_text(text, tokenizer):
    input_data = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
    return input_data


In [12]:
X_val_distilbert = [preprocess_text(text, distilbert_tokenizer) for text in X_val]
X_val_xlnet = [preprocess_text(text, xlnet_tokenizer) for text in X_val]


In [13]:
X_val_distilbert

[{'input_ids': tensor([[  101,  7661,  3177, 20993,  1116, 15701, 12859,  1643, 10205,  1162,
          10605,  1389, 14856, 10205,  9400,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101, 26660, 12420,  2428, 15740,  8499,  1783,  4831,  6867,  1615,
          11945, 24580,  7879,  4693,  2304, 19569, 13703,  1116,  7012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101,  1287, 11679, 15762,  7268,  1116,  1370,   789,  5096,  2217,
          12734,  1158,  2453,  2627,  8652, 11795, 10208,  1507, 11300, 16461,
            102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  101, 18893, 22234,   142,  2050, 15774, 23009, 15974,   156, 11741,
           3408, 16944,  1116,  1585, 10060,  1116,  3279, 10060,  1116,   145,
           2069,  5311,  1545,  4288,  111

In [14]:
X_val_xlnet

[{'input_ids': tensor([[ 1413, 21853,    23, 15612,  5348, 22644, 15111,  4229,  6752, 10200,
              4,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  322, 21310, 25017, 13679,   761,  5246,  4958,  1016,  7386, 13036,
           6360,  2153,   893, 23540,    23,  5899,     4,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[  408, 17020,  2461,  3373,    23,   247,   221,  5370,    23, 11941,
           1203,  2696,  3501,  8202,   703,    23,  8861,   940,  3390, 14448,
              4,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tenso

In [15]:
# Inference
def get_predictions(model, input_data):
    with torch.no_grad():
        output = model(**input_data)
    logits = output.logits
    probabilities = torch.softmax(logits, dim=1)
    return probabilities


In [16]:
distilbert_predictions = [get_predictions(distilbert_model, data) for data in X_val_distilbert]
xlnet_predictions = [get_predictions(xlnet_model, data) for data in X_val_xlnet]


In [17]:
distilbert_predictions

[tensor([[1.8142e-05, 9.9998e-01]]),
 tensor([[9.9999e-01, 1.1322e-05]]),
 tensor([[9.9993e-01, 6.6828e-05]]),
 tensor([[1.0400e-05, 9.9999e-01]]),
 tensor([[9.9998e-01, 1.8702e-05]]),
 tensor([[9.9940e-01, 6.0081e-04]]),
 tensor([[0.0227, 0.9773]]),
 tensor([[2.4701e-05, 9.9998e-01]]),
 tensor([[2.0601e-05, 9.9998e-01]]),
 tensor([[1.4705e-05, 9.9999e-01]]),
 tensor([[0.0060, 0.9940]]),
 tensor([[1.0438e-05, 9.9999e-01]]),
 tensor([[9.9997e-01, 2.7674e-05]]),
 tensor([[9.9959e-01, 4.0715e-04]]),
 tensor([[1.1026e-05, 9.9999e-01]]),
 tensor([[9.9999e-01, 1.3105e-05]]),
 tensor([[5.8572e-05, 9.9994e-01]]),
 tensor([[9.9999e-01, 1.4460e-05]]),
 tensor([[1.4765e-05, 9.9999e-01]]),
 tensor([[2.8967e-05, 9.9997e-01]]),
 tensor([[1.1911e-05, 9.9999e-01]]),
 tensor([[9.9998e-01, 1.7680e-05]]),
 tensor([[9.9998e-01, 1.7475e-05]]),
 tensor([[9.9999e-01, 1.0377e-05]]),
 tensor([[1.1232e-05, 9.9999e-01]]),
 tensor([[1.9798e-05, 9.9998e-01]]),
 tensor([[1.2757e-05, 9.9999e-01]]),
 tensor([[1.0664e

In [18]:
xlnet_predictions

[tensor([[0.0218, 0.9782]]),
 tensor([[0.9962, 0.0038]]),
 tensor([[0.9974, 0.0026]]),
 tensor([[0.0099, 0.9901]]),
 tensor([[0.9308, 0.0692]]),
 tensor([[0.7421, 0.2579]]),
 tensor([[0.8509, 0.1491]]),
 tensor([[0.0083, 0.9917]]),
 tensor([[0.0242, 0.9758]]),
 tensor([[0.0139, 0.9861]]),
 tensor([[0.0302, 0.9698]]),
 tensor([[0.0146, 0.9854]]),
 tensor([[0.9977, 0.0023]]),
 tensor([[0.7407, 0.2593]]),
 tensor([[0.0098, 0.9902]]),
 tensor([[0.9977, 0.0023]]),
 tensor([[0.0332, 0.9668]]),
 tensor([[0.9926, 0.0074]]),
 tensor([[0.0139, 0.9861]]),
 tensor([[0.1313, 0.8687]]),
 tensor([[0.0402, 0.9598]]),
 tensor([[0.9914, 0.0086]]),
 tensor([[0.9969, 0.0031]]),
 tensor([[0.8283, 0.1717]]),
 tensor([[0.0090, 0.9910]]),
 tensor([[0.0159, 0.9841]]),
 tensor([[0.0098, 0.9902]]),
 tensor([[0.0095, 0.9905]]),
 tensor([[0.9943, 0.0057]]),
 tensor([[0.0132, 0.9868]]),
 tensor([[0.9830, 0.0170]]),
 tensor([[0.9981, 0.0019]]),
 tensor([[0.9801, 0.0199]]),
 tensor([[0.9887, 0.0113]]),
 tensor([[0.97

In [19]:
def ensemble_predictions(predictions1, predictions2):
    ensemble_probs = (predictions1 + predictions2) / 2.0  # Simple average ensemble
    return ensemble_probs

ensembled_probabilities = [ensemble_predictions(distilbert_probs, xlnet_probs) for distilbert_probs, xlnet_probs in zip(distilbert_predictions, xlnet_predictions)]


In [20]:
final_predictions = [torch.argmax(ensemble_probs, dim=1).item() for ensemble_probs in ensembled_probabilities]

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_val, final_predictions)
precision = precision_score(y_val, final_predictions)
recall = recall_score(y_val, final_predictions)
f1 = f1_score(y_val, final_predictions)

In [22]:
print(f"Accuracy: {accuracy*100:.4f}")
print(f"Precision: {precision*100:.4f}")
print(f"Recall: {recall*100:.4f}")
print(f"F1 Score: {f1*100:.4f}")

Accuracy: 85.8491
Precision: 87.3016
Recall: 88.7097
F1 Score: 88.0000


In [23]:
import numpy as np

def majority_vote(predictions1, predictions2):
    predictions1 = [np.argmax(probs, axis=1) for probs in predictions1]
    predictions2 = [np.argmax(probs, axis=1) for probs in predictions2]
    ensemble_predictions = [int(np.median([p1, p2])) for p1, p2 in zip(predictions1, predictions2)]
    return ensemble_predictions


In [24]:
# Convert PyTorch tensors to NumPy arrays
distilbert_predictions_np = [probs.numpy() for probs in distilbert_predictions]
xlnet_predictions_np = [probs.numpy() for probs in xlnet_predictions]


In [25]:
# Perform majority vote ensembling
ensemble_predictions = majority_vote(distilbert_predictions_np, xlnet_predictions_np)

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_val, ensemble_predictions)
precision = precision_score(y_val, ensemble_predictions)
recall = recall_score(y_val, ensemble_predictions)
f1 = f1_score(y_val, ensemble_predictions)

In [27]:
print(f"Accuracy: {accuracy*100:.4f}")
print(f"Precision: {precision*100:.4f}")
print(f"Recall: {recall*100:.4f}")
print(f"F1 Score: {f1*100:.4f}")

Accuracy: 84.9057
Precision: 89.6552
Recall: 83.8710
F1 Score: 86.6667


In [28]:
def weighted_ensemble(predictions1, predictions2, weight_model1, weight_model2):
    predictions1 = [np.argmax(probs, axis=1) for probs in predictions1]
    predictions2 = [np.argmax(probs, axis=1) for probs in predictions2]
    ensemble_probs = [int((weight_model1 * p1 + weight_model2 * p2) / (weight_model1 + weight_model2)) for p1, p2 in zip(predictions1, predictions2)]
    return ensemble_probs


In [29]:
# Define the weights for each model
weight_model1 = 0.4
weight_model2 = 0.6


In [30]:
# Perform  weight ensembling
ensemble_predictions_weight = weighted_ensemble(distilbert_predictions_np, xlnet_predictions_np,weight_model1,weight_model2)

In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_val, ensemble_predictions_weight)
precision = precision_score(y_val, ensemble_predictions_weight)
recall = recall_score(y_val, ensemble_predictions_weight)
f1 = f1_score(y_val, ensemble_predictions_weight)

In [34]:
print(f"Accuracy: {accuracy*100:.4f}")
print(f"Precision: {precision*100:.4f}")
print(f"Recall: {recall*100:.4f}")
print(f"F1 Score: {f1*100:.4f}")

Accuracy: 84.9057
Precision: 89.6552
Recall: 83.8710
F1 Score: 86.6667


In [42]:
# Define the weights for each model
weight_model1 = 0.8
weight_model2 = 0.2


In [43]:
# Perform  weight ensembling
ensemble_predictions_weight = weighted_ensemble(distilbert_predictions_np, xlnet_predictions_np,weight_model1,weight_model2)

In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_val, ensemble_predictions_weight)
precision = precision_score(y_val, ensemble_predictions_weight)
recall = recall_score(y_val, ensemble_predictions_weight)
f1 = f1_score(y_val, ensemble_predictions_weight)

In [45]:
print(f"Accuracy: {accuracy*100:.4f}")
print(f"Precision: {precision*100:.4f}")
print(f"Recall: {recall*100:.4f}")
print(f"F1 Score: {f1*100:.4f}")

Accuracy: 84.9057
Precision: 89.6552
Recall: 83.8710
F1 Score: 86.6667


# Stacking

In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.4 MB/s eta 0:00:00


In [ ]:
pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.4 MB/s eta 0:00:00


In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from transformers import DistilBertForSequenceClassification, XLNetForSequenceClassification, DistilBertTokenizer, XLNetTokenizer
from sklearn.linear_model import LogisticRegression
import torch
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted


class TransformersWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, model_name_or_path):
        self.model_name_or_path = model_name_or_path

    def fit(self, X, y):
        # Tokenize and preprocess the input data
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
        model = DistilBertForSequenceClassification.from_pretrained(self.model_name_or_path)

        X_encoded = tokenizer(X, padding=True, truncation=True, return_tensors='pt')
        input_ids = X_encoded['input_ids']

        # Convert labels to tensor
        y_tensor = torch.tensor(y)

        # Train the model (this may vary depending on your specific model and training process)
        outputs = model(input_ids, labels=y_tensor)
        loss, logits = outputs.loss, outputs.logits

        self.model = model

        return self

    def predict(self, X):
        # Check if the model is fitted
        check_is_fitted(self, 'model')

        # Tokenize and preprocess the input data
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-cased')
        X_encoded = tokenizer(X, padding=True, truncation=True, return_tensors='pt')
        input_ids = X_encoded['input_ids']

        # Make predictions
        with torch.no_grad():
            logits = self.model(input_ids).logits

        # Convert logits to class labels (assuming binary classification)
        predicted_labels = torch.argmax(logits, dim=1)

        return predicted_labels


In [ ]:
distilbert_model = TransformersWrapper("/content/gdrive/MyDrive/fake_news/Models/DistilBERT/ds_35_h3")

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted


class TransformersWrapperXL(BaseEstimator, ClassifierMixin):
    def __init__(self, model_name_or_path):
        self.model_name_or_path = model_name_or_path

    def fit(self, X, y):
        # Tokenize and preprocess the input data
        tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
        model = XLNetForSequenceClassification.from_pretrained(self.model_name_or_path)

        X_encoded = tokenizer(X, padding=True, truncation=True, return_tensors='pt')
        input_ids = X_encoded['input_ids']

        # Convert labels to tensor
        y_tensor = torch.tensor(y)

        # Train the model (this may vary depending on your specific model and training process)
        outputs = model(input_ids, labels=y_tensor)
        loss, logits = outputs.loss, outputs.logits

        self.model = model

        return self

    def predict(self, X):
        # Check if the model is fitted
        check_is_fitted(self, 'model')

        # Tokenize and preprocess the input data
        tokenizer = XLNetTokenizer.from_pretrained('distilbert-base-cased')
        X_encoded = tokenizer(X, padding=True, truncation=True, return_tensors='pt')
        input_ids = X_encoded['input_ids']

        # Make predictions
        with torch.no_grad():
            logits = self.model(input_ids).logits

        # Convert logits to class labels (assuming binary classification)
        predicted_labels = torch.argmax(logits, dim=1)

        return predicted_labels


In [ ]:
xlnet_model = TransformersWrapperXL("/content/gdrive/MyDrive/fake_news/Models/XLNet/ds_35")

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/fake_news/dfcleaned.csv")
df['SECTION_CLEANED'] = df['SECTION_CLEANED'].astype(str)

# Randomly select  % of the data
df_data = df.sample(frac=0.35, random_state=42)

In [ ]:
labels = list(df_data['label'])
len(labels)

13526

In [ ]:
labels = [0 if label == "FAKE" else 1 for label in labels]
len(labels)

13526

In [ ]:
# Assuming you have labeled data for training and testing
X_train, X_test, y_train, y_test = train_test_split(list(df_data['SECTION_CLEANED']), labels, test_size=0.2,random_state=42)

In [ ]:
base_models = [
    ('well_trained_model', distilbert_model),
    ('overfitting_model', xlnet_model)
]

In [ ]:
meta_learner = LogisticRegression()

In [ ]:
stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_learner)


In [ ]:
stacking_model.fit(X_train, y_train)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [ ]:
y_pred = stacking_model.predict(X_test)

In [ ]:
# Calculate accuracy

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)



In [ ]:
# Generate a classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)